In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# This script loads a custom 96w deep well plate 2.2mL v-bottom and 
# Tests the dimensions of the plate so I don't have to repeatedly run lh.setup

# ── imports ──────────────────────────────────────────────
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import STARBackend
from pylabrobot.resources.hamilton import (
    STARLetDeck,
    MFX_CAR_L5_base,
    TIP_CAR_480_A00
)
from pylabrobot.resources.hamilton.mfx_modules import (
    MFX_DWP_module_188042,
    MFX_DWP_rackbased_module
)
from pylabrobot.resources import (
    TIP_50ul_w_filter,
                 HTF             # 50 µL filter tip rack
)

from pylabrobot.resources.corning.plates import Cor_96_wellplate_360ul_Fb

import asyncio

# ── build LH + deck ──────────────────────────────────────
backend = STARBackend()
lh      = LiquidHandler(backend=backend, deck=STARLetDeck())

# ── initialise hardware (Autoload) ───────────────────────
await lh.setup(skip_autoload=True)

# ── tip carrier with 50 µL tips ───────────────────────────
tip_car = TIP_CAR_480_A00("tip_car")
tip_car[0] = HTF(name="tips_00")
tip_car[1] = TIP_50ul_w_filter(name="tips_01")
lh.deck.assign_child_resource(tip_car, rails=1)


# ── module → carrier → deck ──────────────────────────────
dwp_mod   = MFX_DWP_module_188042("dwp_mod_1")
other_mod = MFX_DWP_rackbased_module("dwp_mod_2")
flex_car  = MFX_CAR_L5_base("flex_car_1", modules={0: dwp_mod, 1: other_mod})
lh.deck.assign_child_resource(flex_car, rails=13)

In [3]:
# ── pick up a single 50 µL tip on channel 3 ─────────────
tiprack = lh.deck.get_resource("tips_01")
longrack = lh.deck.get_resource("tips_00")
# await lh.pick_up_tips(longrack["B4"], use_channels=[5])
await lh.pick_up_tips(tiprack["A1"], use_channels=[5])

In [ ]:
from typing import Optional

from pylabrobot.resources.height_volume_functions import (
  compute_height_from_volume_rectangle,
  compute_volume_from_height_rectangle,
)
from pylabrobot.resources.plate import Lid, Plate
from pylabrobot.resources.utils import create_ordered_items_2d
from pylabrobot.resources.well import (
  CrossSectionType,
  Well,
  WellBottomType,
)

def BioER_96_wellplate_Vb_2200ul(name: str, lid: Optional[Lid] = None) -> Plate:
  """
  BioER plate similar to NEST and other KingFisher-compatible deep well plates
  https://en.bioer.com/uploadfiles/2024/05/20240513165756879.pdf
  Cat no. BSH06M1T-A
  """
  INNER_WELL_WIDTH = 8.2  # from spec  
  INNER_WELL_LENGTH = 8.2  # from spec

  well_kwargs = {
    "size_x": INNER_WELL_WIDTH,  # measured
    "size_y": INNER_WELL_LENGTH,  # measured
    "size_z": 42.4,  # from spec
    "bottom_type": WellBottomType.V,
    "cross_section_type": CrossSectionType.RECTANGLE,
    "compute_height_from_volume": lambda liquid_volume: compute_height_from_volume_rectangle(
      liquid_volume,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    "compute_volume_from_height": lambda liquid_height: compute_volume_from_height_rectangle(
      liquid_height,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    # "material_z_thickness": 1,
    "material_z_thickness": 0.80, # measured
  }

  return Plate(
    name=name,
    size_x=127.30,  # from spec
    size_y=85.20,  # from spec
    size_z=44.2,  # from spec
    lid=lid,
    model=BioER_96_wellplate_Vb_2200ul.__name__,
    ordered_items=create_ordered_items_2d(
      Well,
      num_items_x=12,
      num_items_y=8,
      dx=9.5,  # measured
      dy=9.5,  # measured
      dz=1.2,  # from spec
      item_dx=9, # from spec
      item_dy=9, # from spec
      **well_kwargs,
    ),
  )

def opentrons_24_tuberack_generic_1point5ml_snapcap_short(name: str, lid: Optional[Lid] = None) -> Plate:
  """
  OpenTrons 24 well rack with the shorter stand
  3D print available here: https://www.thingiverse.com/thing:3405002
  Spec sheet (json):
  https://raw.githubusercontent.com/Opentrons/opentrons/edge/shared-data/labware/definitions/2/opentrons_24_tuberack_nest_1.5ml_screwcap/1.json
  """
  INNER_WELL_WIDTH = 9.2  # measured  
  INNER_WELL_LENGTH = 9.2  # measured

  well_kwargs = {
    "size_x": INNER_WELL_WIDTH,  # measured
    "size_y": INNER_WELL_LENGTH,  # measured
    "size_z": 37.40,  # measured
    "bottom_type": WellBottomType.V,
    "cross_section_type": CrossSectionType.RECTANGLE,
    "compute_height_from_volume": lambda liquid_volume: compute_height_from_volume_rectangle(
      liquid_volume,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    "compute_volume_from_height": lambda liquid_height: compute_volume_from_height_rectangle(
      liquid_height,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    # "material_z_thickness": 1,
    "material_z_thickness": 0.80, # measured
  }

  return Plate(
    name=name,
    size_x=127.75,  # from spec
    size_y=85.50,  # from spec
    size_z=48.5,  # measured (this is the shorter platform)
    lid=lid,
    model=opentrons_24_tuberack_generic_1point5ml_snapcap_short.__name__,
    ordered_items=create_ordered_items_2d(
      Well,
      num_items_x=6,
      num_items_y=4,
      dx=12.5,  # measured
      dy=16.5,  # measured
      dz=18,  # measured
      item_dx=19.89, # from spec
      item_dy=19.28, # from spec
      **well_kwargs,
    ),
  )

from pylabrobot.resources.height_volume_functions import (
  calculate_liquid_height_in_container_2segments_square_vbottom,
  calculate_liquid_volume_container_2segments_square_vbottom,
)
from pylabrobot.resources.plate import Lid, Plate
from pylabrobot.resources.utils import create_ordered_items_2d
from pylabrobot.resources.well import (
  CrossSectionType,
  Well,
  WellBottomType,
)

def VWR_96_wellplate_100_Vb(name: str, with_lid: bool = False) -> Plate:
  """
This plate is a VWR PCR plate 96 well low-profile, half-skirted, ABI-FAST type plate.
VWR cat no. 89218-296
It is half-skirted so it must reside in another plate like a Cor_96_wellplate_360ul_Fb
  """
  
  return Plate(
    name=name,
    size_x=127.76,
    size_y=85.48,
    size_z=20.0,
    # lid=lid,
    model=VWR_96_wellplate_100_Vb.__name__,
    ordered_items=create_ordered_items_2d(
      Well,
      num_items_x=12,
      num_items_y=8,
      dx=10.25,  # keeping costar measurement
      dy=11.0,  # 7.77 keeping costar measurement
      dz=8.5, # how high is well above base
      item_dx=9.0,
      item_dy=9.0,
      size_x=5.4,  # measured
      size_y=5.4,  # measured
      size_z=16.3, # measured well depth, costar + VWR plate height
      material_z_thickness=0.5,
      bottom_type=WellBottomType.V,
      cross_section_type=CrossSectionType.CIRCLE,
      max_volume=100,
    ),
  )

In [56]:
# await lh.pick_up_tips(tiprack["B1"], use_channels=[5])
# await lh.pick_up_tips(longrack["B4"], use_channels=[5])
plate19 = VWR_96_wellplate_100_Vb(name="plate19")
dwp_mod.assign_child_resource(plate19)
plate = lh.deck.get_resource("plate19")
liqH = 0

await lh.prepare_for_manual_channel_operation(3)
await lh.aspirate(plate["A1"], vols=[0], liquid_height=[liqH], use_channels=[5])
# await lh.prepare_for_manual_channel_operation(3)
await lh.aspirate(plate["H1"], vols=[0], liquid_height=[liqH], use_channels=[5])
# await lh.prepare_for_manual_channel_operation(3)
await lh.aspirate(plate["A12"], vols=[0], liquid_height=[liqH], use_channels=[5])
# await lh.prepare_for_manual_channel_operation(3)
await lh.aspirate(plate["H12"], vols=[0], liquid_height=[liqH], use_channels=[5])
await lh.prepare_for_manual_channel_operation(3)

# # await lh.aspirate(plate["H1"], vols=[0], liquid_height=[3], use_channels=[3])
# await lh.aspirate(plate["H1"], vols=[0], use_channels=[3])



In [ ]:
# await lh.drop_tips(longrack["B4"], use_channels=[5])
# await lh.drop_tips(tiprack["A1"], use_channels=[5])
# await lh.discard_tips()
# await lh.stop()